In [63]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

C:\Users\Dan\AppData\Local\Temp\ipykernel_26116\3777615979.py:1: DeprecationWarning: Importing display from IPython.core.display is deprecated since IPython 7.14, please import from IPython display
  from IPython.core.display import display, HTML


# Lab | Natural Language Processing
### SMS: SPAM or HAM

### Let's prepare the environment

In [64]:
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.feature_extraction.text import TfidfVectorizer

- Read Data for the Fraudulent Email Kaggle Challenge
- Reduce the training set to speead up development. 

In [65]:
## Read Data for the Fraudulent Email Kaggle Challenge
data = pd.read_csv(r"../data/kg_train.csv",encoding='latin-1')

# Reduce the training set to speed up development. 
# Modify for final system
data = data.head(1000)
print(data.shape)
data.fillna("",inplace=True)

(1000, 2)


### Let's divide the training and test set into two partitions

In [66]:
from sklearn.model_selection import train_test_split


X = data.drop(columns=['text'])
y = data['label']

# Split data into training and testing sets (80% train, 20% test)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


## Data Preprocessing

In [67]:
import string
from nltk.corpus import stopwords
print(string.punctuation)
print(stopwords.words("english")[100:110])
from nltk.stem.snowball import SnowballStemmer
snowball = SnowballStemmer('english')

!"#$%&'()*+,-./:;<=>?@[\]^_`{|}~
['here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each']


## Now, we have to clean the html code removing words

- First we remove inline JavaScript/CSS
- Then we remove html comments. This has to be done before removing regular tags since comments can contain '>' characters
- Next we can remove the remaining tags

- Remove all the special characters
    
- Remove numbers
    
- Remove all single characters
 
- Remove single characters from the start

- Substitute multiple spaces with single space

- Remove prefixed 'b'

- Convert to Lowercase

In [68]:
import re
from bs4 import BeautifulSoup
import nltk

from nltk.corpus import stopwords

nltk.download('stopwords')
# english_stopwords = set(stopwords.words("english")[100:110])
english_stopwords = set(stopwords.words('english'))


# this function does all the cleaning, including removing stopwords as instructed 
def clean_text(text, stop_words=None):
    # Remove inline JavaScript/CSS
    text = re.sub(r'<(script|style).*?>.*?(</\1>)', '', text, flags=re.DOTALL)
    # Remove HTML comments
    text = re.sub(r'<!--.*?-->', '', text, flags=re.DOTALL)
    # Remove remaining HTML tags
    text = re.sub(r'<[^>]+>', '', text)
    # Remove all special characters
    text = re.sub(r'[^A-Za-z\s]', '', text)
    # Remove numbers
    text = re.sub(r'\d+', '', text)
    # Remove all single characters
    text = re.sub(r'\b\w\b', '', text)
    # Substitute multiple spaces with a single space
    text = re.sub(r'\s+', ' ', text)
    # Convert to lowercase
    text = text.lower()
    # Remove stopwords if provided
    if stop_words:
        text = ' '.join(word for word in text.split() if word not in stop_words)
    return text

columns_to_clean = ['text']  
for col in columns_to_clean:
    data[col] = data[col].apply(lambda x: clean_text(x, stop_words=english_stopwords))




[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Dan\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


## Now let's work on removing stopwords


In [ ]:
import re
from bs4 import BeautifulSoup
import nltk

from nltk.corpus import stopwords

nltk.download('stopwords')
# english_stopwords = set(stopwords.words("english")[100:110])
english_stopwords = set(stopwords.words('english'))


# this function does all the cleaning, including removing stopwords as instructed 
def clean_text(text, stop_words=None):
    # Remove inline JavaScript/CSS
    text = re.sub(r'<(script|style).*?>.*?(</\1>)', '', text, flags=re.DOTALL)
    # Remove HTML comments
    text = re.sub(r'<!--.*?-->', '', text, flags=re.DOTALL)
    # Remove remaining HTML tags
    text = re.sub(r'<[^>]+>', '', text)
    # Remove all special characters
    text = re.sub(r'[^A-Za-z\s]', '', text)
    # Remove numbers
    text = re.sub(r'\d+', '', text)
    # Remove all single characters
    text = re.sub(r'\b\w\b', '', text)
    # Substitute multiple spaces with a single space
    text = re.sub(r'\s+', ' ', text)
    # Convert to lowercase
    text = text.lower()
    # Remove stopwords if provided
    if stop_words:
        text = ' '.join(word for word in text.split() if word not in stop_words)
    return text

columns_to_clean = ['text']  
for col in columns_to_clean:
    data[col] = data[col].apply(lambda x: clean_text(x, stop_words=english_stopwords))




## Tame Your Text with Lemmatization
Break sentences into words, then use lemmatization to reduce them to their base form (e.g., "running" becomes "run"). See how this creates cleaner data for analysis!

In [69]:
from nltk.stem import WordNetLemmatizer

nltk.download('wordnet')
nltk.download('omw-1.4')

lemmatizer = WordNetLemmatizer()

def tokenize_and_lemmatize(text, stop_words=None):
    # Tokenize text into words
    words = text.split()
    # Remove stopwords and apply lemmatization
    if stop_words:
        words = [lemmatizer.lemmatize(word) for word in words if word not in stop_words]
    else:
        words = [lemmatizer.lemmatize(word) for word in words]
    # Join words back into a single string
    return ' '.join(words)

columns_to_clean = ['text']  
for col in columns_to_clean:
    data[col] = data[col].apply(lambda x: tokenize_and_lemmatize(x, stop_words=english_stopwords))



[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\Dan\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to
[nltk_data]     C:\Users\Dan\AppData\Roaming\nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


In [70]:
from sklearn.model_selection import train_test_split

# Define features and target
X = data[['text']] 
y = data['label']  

# Split into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


## Bag Of Words
Let's get the 10 top words in ham and spam messages (**EXPLORATORY DATA ANALYSIS**)

In [71]:
from collections import Counter

# Combine all rows in the 'text' column into a single string
all_text = ' '.join(data['text'])

# Split the combined text into individual words
words = all_text.split()

# Count the frequency of each word
word_counts = Counter(words)

# Get the 10 most common words
top_10_words = word_counts.most_common(10)

# Print the results
print("Top 10 Words:", top_10_words)

Top 10 Words: [('money', 924), ('account', 800), ('bank', 754), ('fund', 704), ('u', 665), ('business', 496), ('country', 457), ('transaction', 416), ('million', 396), ('transfer', 393)]


## Extra features

In [72]:
# We add to the original dataframe two additional indicators (money symbols and suspicious words).
money_simbol_list = "|".join(["euro","dollar","pound","€","$"])
suspicious_words = "|".join(["free","cheap","sex","money","account","bank","fund","transfer","transaction","win","deposit","password"])

# Add indicators to the training set
X_train['money_mark'] = X_train['text'].str.contains(money_simbol_list) * 1
X_train['suspicious_words'] = X_train['text'].str.contains(suspicious_words) * 1
X_train['text_len'] = X_train['text'].apply(lambda x: len(x))

# Add indicators to the testing set
X_test['money_mark'] = X_test['text'].str.contains(money_simbol_list) * 1
X_test['suspicious_words'] = X_test['text'].str.contains(suspicious_words) * 1
X_test['text_len'] = X_test['text'].apply(lambda x: len(x))

data.head()

,text,label
0,dear sir strictly private business proposal mi...,1
1,,0
2,noracheryl emailed dozen memo haiti weekend pl...,0
3,dear sirfmadamc know proposal might surprise e...,1
4,fyi,0


## How would work the Bag of Words with Count Vectorizer concept?

In [82]:
# Vectorization:

# Each text is transformed into a vector where element corresponds to a word in the vocabulary.
# The value represents the count of how many times the word appears in the text.

from sklearn.feature_extraction.text import CountVectorizer

# Initialize CountVectorizer
vectorizer = CountVectorizer()

# Fit the vectorizer and transform the text
X = vectorizer.fit_transform(words)

# Convert to array for better visualization
print(X.toarray())

# Get the vocabulary (mapping of words to indices)
print(vectorizer.vocabulary_)



[[0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 ...
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]]
{'dear': 4536, 'sir': 15609, 'strictly': 16157, 'private': 13513, 'business': 2724, 'proposal': 13692, 'mike': 10747, 'chukwu': 3258, 'manager': 10334, 'bill': 2364, 'exchange': 6141, 'foreign': 6800, 'remittance': 14369, 'department': 4744, 'zenith': 19325, 'international': 8870, 'bank': 1904, 'plc': 13119, 'writing': 19022, 'letter': 9844, 'ask': 1483, 'support': 16327, 'cooperation': 4073, 'carry': 2967, 'opportunity': 12386, 'discovered': 5098, 'abandoned': 19, 'sum': 16291, 'fifteen': 6562, 'million': 10765, 'united': 17988, 'state': 16038, 'dollar': 5253, 'account': 167, 'belongs': 2186, 'one': 12286, 'customer': 4369, 'died': 4989, 'along': 781, 'entire': 5909, 'family': 6369, 'wife': 18768, 'two': 17817, 'child': 3207, 'november': 11864, 'plane': 13091, 'crash': 4278, 'since': 15552, 'heard': 7783, 'death': 4550, 'expecting': 6200, 'nextofkin': 11638, 'come': 

## TD-IDF

- Load the vectorizer

- Vectorize all dataset

- print the shape of the vetorized dataset

In [74]:
# TF-IDF (Term Frequency-Inverse Document Frequency) assigns weights to words based on their importance. 

from sklearn.feature_extraction.text import TfidfVectorizer

vectorizer = TfidfVectorizer()

# Fit the vectorizer and transform the text
X = vectorizer.fit_transform(words)


print(X.shape)

(97681, 19352)


## And the Train a Classifier?

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, accuracy_score
import joblib


# Initialize TF-IDF Vectorizer
vectorizer = TfidfVectorizer()


# Fit and transform  training data with TF-IDF
X_train_tfidf = vectorizer.fit_transform(X_train['text'])

# Transform  testing data using same vectorizer
X_test_tfidf = vectorizer.transform(X_test['text'])

# I will use Logistic regression as classifier
classifier = LogisticRegression(random_state=42)

# Training  classifier on TF-IDF features
classifier.fit(X_train_tfidf, y_train)

# Make predictions on  test set
y_pred = classifier.predict(X_test_tfidf)

# Print results
print("Accuracy:", accuracy_score(y_test, y_pred))
print("Classification Report:\n", classification_report(y_test, y_pred))


# saving trained model and TF-IDF vectorizer 
joblib.dump(classifier, "classifier_model.pkl")
joblib.dump(vectorizer, "tfidf_vectorizer.pkl")

Accuracy: 0.96
Classification Report:
               precision    recall  f1-score   support

           0       0.94      1.00      0.97       125
           1       1.00      0.89      0.94        75

    accuracy                           0.96       200
   macro avg       0.97      0.95      0.96       200
weighted avg       0.96      0.96      0.96       200



In [78]:
# trying GridSearchCV to optimize logistic regressoin parameters

from sklearn.model_selection import GridSearchCV

param_grid = {'C': [0.1, 1, 10], 'solver': ['liblinear', 'lbfgs']}
grid_search = GridSearchCV(LogisticRegression(random_state=42), param_grid, cv=5, scoring='accuracy')
grid_search.fit(X_train_tfidf, y_train)
print("Best parameters:", grid_search.best_params_)


Best parameters: {'C': 10, 'solver': 'liblinear'}


In [80]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, accuracy_score



columns_to_clean = ['text']  
for col in columns_to_clean:
    data[col] = data[col].apply(lambda x: clean_text(x, stop_words=english_stopwords))

# Train the model with  best parameters
best_classifier = LogisticRegression(C=10, solver='liblinear', random_state=42)
best_classifier.fit(X_train_tfidf, y_train)

# Evaluate the model
y_pred_best = best_classifier.predict(X_test_tfidf)

# Print results
print("Optimized Model Accuracy:", accuracy_score(y_test, y_pred_best))
print("Optimized Model Classification Report:\n", classification_report(y_test, y_pred_best))


Optimized Model Accuracy: 0.975
Optimized Model Classification Report:
               precision    recall  f1-score   support

           0       0.96      1.00      0.98       125
           1       1.00      0.93      0.97        75

    accuracy                           0.97       200
   macro avg       0.98      0.97      0.97       200
weighted avg       0.98      0.97      0.97       200



In [79]:
# what words are more triggering?

feature_names = vectorizer.get_feature_names_out()
coefficients = classifier.coef_[0]
sorted_features = sorted(zip(coefficients, feature_names), reverse=True)
print("Top features:", sorted_features[:10])


Top features: [(2.6535764860141318, 'money'), (2.312596083072694, 'fund'), (2.2813028114681755, 'bank'), (2.0559884125900796, 'account'), (1.6059625698220503, 'company'), (1.4744634774707235, 'business'), (1.4290916897836037, 'transaction'), (1.3727621126342513, 'contact'), (1.3355644482054387, 'transfer'), (1.21210512190839, 'country')]


### Extra Task - Implement a SPAM/HAM classifier

https://www.kaggle.com/t/b384e34013d54d238490103bc3c360ce

The classifier can not be changed!!! It must be the MultinimialNB with default parameters!

Your task is to find the **best feature representation**.

You can work with teams of two persons (recommended).

In [ ]:
# The evaluation metric for this competition is Mean F1-Score. 
# MultinimialNB = Multinomial Naive Bayes

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.naive_bayes import MultinomialNB


## Read Data for the Fraudulent Email Kaggle Challenge
data = pd.read_csv(r"../data/kg_train.csv",encoding='latin-1')

# Reduce the training set to speed up development. 
# Modify for final system
data = data.head(1000)
data.fillna("",inplace=True)


columns_to_clean = ['text']  
for col in columns_to_clean:
    data[col] = data[col].apply(lambda x: clean_text(x, stop_words=english_stopwords))

columns_to_clean = ['text']  
for col in columns_to_clean:
    data[col] = data[col].apply(lambda x: tokenize_and_lemmatize(x, stop_words=english_stopwords))


# Split into features and target
X = data['text']
y = data['label']

# Split into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Initialize CountVectorizer with bigrams
ngram_vectorizer = CountVectorizer(ngram_range=(2, 2))

# Fit and transform training data, transform test data
X_train_ngram = ngram_vectorizer.fit_transform(X_train)
X_test_ngram = ngram_vectorizer.transform(X_test)

# Train and evaluate MultinomialNB
classifier = MultinomialNB()
classifier.fit(X_train_ngram, y_train)
y_pred_ngram = classifier.predict(X_test_ngram)

# Print results
print("N-grams Representation (Bigrams)")
print("Accuracy:", accuracy_score(y_test, y_pred_ngram))
print("Classification Report:\n", classification_report(y_test, y_pred_ngram))



N-grams Representation (Bigrams)
Accuracy: 0.91
Classification Report:
               precision    recall  f1-score   support

           0       0.95      0.90      0.93       125
           1       0.85      0.92      0.88        75

    accuracy                           0.91       200
   macro avg       0.90      0.91      0.91       200
weighted avg       0.91      0.91      0.91       200



The best model seems to be the optimized model fro before. 